# Pre-processing steps

Set input_folders as the path to the main folder the images are stored within. Note that the main folder can be structured so that it contains subfolders of images. At the Missouri Botanical Garden, image file names are specimen barcodes.

In [ ]:
input_folders = [
    "path to folder images are stored in"
]

Set archive_folder as the path to the folder where images should be sent after being processed.

In [ ]:
archive_folder = "path to the folder where images are sent after processing"

This code will save two files: (1) a CSV of transcription results and (2) a tab-delimited text file of transcription results.

Set output_folder as the path to the folder where the CSV and text files of transcription results should be saved.

In [ ]:
output_folder = "path to the folder where CSV and text output is saved"

Set the file names for these two files.

In [ ]:
csv_file_name = "CSV file name"
txt_file_name = "Text file name"

Ensure google-cloud-vision, google-cloud-translate, and openai are installed.

In [ ]:
!pip install --upgrade google-cloud-vision
!pip install google-cloud-translate
!pip install openai

# Step 1: OCR

Load the following libraries.

In [ ]:
from google.cloud import vision 
import json
import cv2
import numpy as np 
import requests
import csv
import os
from langdetect import detect
from google.cloud import translate_v2 as translate
import concurrent.futures
import re
import shutil
from PIL import Image
import io
import concurrent.futures
import pandas as pd

Define the following functions.

In [ ]:
# Write a 'detect_language' function that detects the language of a text file. This function will return 'None' if the language detection fails.
def detect_language(text):
    try:
        language = detect(text)
        return language
    except:
        return None

# Write a function that will remove sequences of numbers that commonly correspond to ruler text.
def remove_specific_number_sequences(text):
    pattern = r'0\s1\s2\s3\s4\s5\s6\s7\s8\s9\s10\s*(cm)?|0\s1\s2\s3\s4\s5\s6\s7\s8\s9\s*(cm\s)?10|0\s1\s2\s3\s4\s5\s6\s7\s8\s*(cm\s)?9\s10|0\s1\s2\s3\s4\s5\s6\s7\s*(cm\s)?8\s9\s10|0\s1\s2\s3\s4\s5\s6\s7\s*(cm\s)?9\s10|0\s1\s2\s3\s4\s5\s6\s7\s*(cm\s)?8\s9\s10|0\s1\s2\s3\s4\s5\s6\s*(cm\s)?7\s8\s9\s10|0\s1\s2\s3\s4\s5\s*(cm\s)?6\s7\s8\s9\s10|0\s1\s2\s3\s4\s*(cm\s)?5\s6\s7\s8\s9\s10|0\s1\s2\s3\s*(cm\s)?4\s5\s6\s7\s8\s9\s10|0\s1\s2\s*(cm\s)?3\s4\s5\s6\s7\s8\s9\s10|0\s1\s*(cm\s)?2\s3\s4\s5\s6\s7\s8\s9\s10|0\s*(cm\s)?1\s2\s3\s4\s5\s6\s7\s8\s9\s10|cm\s0\s1\s2\s3\s4\s5\s6\s7\s8\s9\s10|01\s(cm\s)?1\s2\s3\s4\s5\s6\s7\s8\s9\s10'
    
    cleaned_text = re.sub(pattern, '', text)
    
    return cleaned_text

# Define a list of non-Latin languages that may occur in Asian specimens.
asian_non_latin_languages = [
    'am',  # Amharic
    'ar',  # Arabic
    'bn',  # Bengali
    'gu',  # Gujarati
    'iw',  # Hebrew
    'hi',  # Hindi
    'ja',  # Japanese
    'kn',  # Kannada
    'ko',  # Korean
    'ml',  # Malayalam
    'mr',  # Marathi
    'ru',  # Russian
    'ta',  # Tamil
    'te',  # Telugu
    'th',  # Thai
    'zh-CN',  # Chinese (PRC)
    'zh-TW',  # Chinese (Taiwan)
    'ur',  # Urdu
    'vi'  # Vietnamese
]

# Write a function to detect language and translate if it is within asian_non_latin_languages 
def translate_text(text, dest_lang='en'):
    translate_client = translate.Client()

    # Detect the language of the text
    detected_language = translate_client.detect_language(text)['language']

    # Check if the detected language is in asian_non_latin_languages
    if detected_language in asian_non_latin_languages and detected_language != dest_lang:
        # Translate the content to English
        translation = translate_client.translate(
            text,
            target_language=dest_lang,
        )
        return translation['translatedText']
    else:
        # If the language is not in asian_non_latin_languages or is already English, do not translate
        return text

# Write a 'process_image' function
def process_image(image_path, root):
    """Detects text in images stored in a local folder."""

    # Check if the image has already been processed
    if image_path in processed_images:
        return
    
    # Initialize the Google Cloud Vision client
    client = vision.ImageAnnotatorClient()
    
    try:
        # Set the folder name
        folder_name = os.path.basename(root)
        
        # Read the image file
        with open(image_path, 'rb') as image_file:
            image_content = image_file.read()

        # Convert TIFF to JPEG with compression
        with Image.open(io.BytesIO(image_content)) as img:
            # Convert to RGB
            img = img.convert('RGB')
            
            # Create an in-memory byte stream to hold JPEG image data
            jpeg_image_stream = io.BytesIO()
            
            # Save the JPEG image to the byte stream
            img.save(jpeg_image_stream, format='JPEG', quality=85)
            
            # Get JPEG image content as bytes
            jpeg_image_content = jpeg_image_stream.getvalue()

        # Create a Vision API image object
        image = vision.Image(content=jpeg_image_content)

        # Run text detection
        response = client.text_detection(image=image)
        texts = response.text_annotations

        # Define a regular expression pattern to match "MO-" followed by numbers.
        # For MO specimens, this will locate the specimen barcode.
        pattern = r'MO-\d+\s'
        
        # Iterate through lines of the text
        for text_annotation in texts:
            text_annotation.description = re.sub(pattern, '', text_annotation.description)

        # Create a dictionary to store the results
        results = {"Texts": []}

        for text_annotation in texts:
            text_data = {
                "description": text_annotation.description,
                "bounds": [(vertex.x, vertex.y) for vertex in text_annotation.bounding_poly.vertices]
            }
            results["Texts"].append(text_data)
        
        # Extract the currentBarcodeID from the image_path
        currentBarcodeID = os.path.splitext(os.path.basename(image_path))[0]

        if response.error.message:
            raise Exception(
                "{}\nFor more info on error messages, check: "
                "https://cloud.google.com/apis/design/errors".format(response.error.message)
            )

        # Extract the first description
        first_description = results['Texts'][0]['description']

        # Split the text into segments
        segments = first_description.split('\n') 
        
        # Individually translate each segment
        translated_segments = []
        for segment in segments:
            translated_segment = translate_text(segment, dest_lang='en')
            # Replace line breaks with spaces
            translated_segment = translated_segment.replace('\n', ' ')
            translated_segments.append(translated_segment)
        translated_text = translated_segments

        # Create a single string from the list
        translated_text_str = ' '.join(translated_text)
        
        # Again replace line breaks with spaces
        translated_text = translated_text_str.replace('\n', ' ')

        # Remove the sequences that commonly correspond to ruler text
        translated_text = remove_specific_number_sequences(translated_text)

        # Create a list with currentBarcodeID, folder_name, and translated_text
        image_data = [currentBarcodeID, folder_name, translated_text]

        # Append the current image's data to all_image_data
        all_image_data.append(image_data)

        # Move the processed image to the archive_folder folder
        os.makedirs(archive_folder, exist_ok=True)
        archive_path = os.path.join(archive_folder, os.path.basename(image_path))
        shutil.move(image_path, archive_path)
        print(f"Moved {image_path} to {archive_path}")

        # After processing, add the image path to the set of processed images
        processed_images.add(image_path)

    except Exception as e:
        # Handle any exceptions that may occur during image processing
        print(f"Error processing image {image_path}: {str(e)}")

# Write a function that performs parallel image processing
def detect_text_local_folder(input_folders):
    """Detects text in images stored in local folders."""
    client = vision.ImageAnnotatorClient()
    
    # Track the processed subfolders
    processed_subfolders = set()
    all_image_paths = [] 

    for input_folder in input_folders:
        # Use os.walk to cycle through subdirectories and files
        for root, dirs, files in os.walk(input_folder):
            image_paths = []
            
            for file_name in files:
                if file_name.lower().endswith(('.jpg', '.jpeg', '.tif', '.tiff')):
                    image_path = os.path.join(root, file_name)
                    print(f"Processing image: {image_path} in root: {root}")
                    process_image(image_path, root)
                    image_paths.append(image_path)
            
            # Add the current root directory to processed_subfolders if it contains images
            if image_paths:
                processed_subfolders.add(root)
            
            # Add all image paths
            all_image_paths.extend(image_paths)

    # Perform parallel processing
    with concurrent.futures.ThreadPoolExecutor(max_workers=16) as executor:
        for image_path in all_image_paths:
            print(f"Submitting image processing task for: {image_path}")
            executor.submit(process_image, image_path, os.path.dirname(image_path)) 

    # Move all empty subfolders in input_folders to the archive_folder
    for input_folder in input_folders:
        for root, dirs, _ in os.walk(input_folder):
            if root not in processed_subfolders and not dirs and not os.listdir(root):
                subfolder_to_move = root
                archive_subfolder_path = os.path.join(archive_folder, os.path.relpath(subfolder_to_move, input_folder))
                
                if not os.path.exists(archive_subfolder_path):
                    shutil.move(subfolder_to_move, archive_subfolder_path)
                    print(f"Moved empty subfolder {subfolder_to_move} to {archive_subfolder_path}")


Initialize processed_images to keep track of processed image paths.

In [ ]:
processed_images = set()

Initialize an empty list to collect data for each image. The column headers are set here.

In [ ]:
all_image_data = [["barcodeID", "storedUnder", "translatedText"]]

Run detect_text_local_folder(input_folder). This will loop across all images within the input_folder path.

In [ ]:
detect_text_local_folder(input_folders)

Check to see whether all_image_data was created successfully.

In [ ]:
all_image_data

# Step 2: ChatGPT

Load the following libraries and set the OpenAI API key.

In [ ]:
import os
import pandas as pd
from openai import OpenAI

# Set client as your unique OpenAI API key.
client = OpenAI(
    api_key = "API key"
)

Save all_image_data as HerbariumSpecimenData data frame.

In [ ]:
HerbariumSpecimenData = pd.DataFrame(all_image_data[1:], columns=all_image_data[0])

Create a data frame for storing the results from the following loop.

In [ ]:
results = pd.DataFrame(columns=['barcodeID', 'AccessionNumber', 'Continent', 'Country', 'LocalityDescription', 
                                'Elevation', 'CollectionDay', 'CollectionMonth', 'CollectionYear'])

Run a loop that uses OpenAI's GPT-3.5 Turbo model to extract data from specimen text for transcriptions. This loop will process all specimens in HerbariumSpecimenData.

In [ ]:
for index, row in HerbariumSpecimenData.iterrows():
    
    ## Store the barcode and specimen text as objects.
    barcode_id = row['barcodeID']
    translated_text = row['translatedText']
    
    ## Accession Number
    accessionNumber = client.chat.completions.create(
        messages = [
            {
                "role": "user",
                "content": f"Here is label text from an herbarium specimen: '{translated_text}'. What is the accession number of this herbarium specimen? Return only the number and no extraneous text.",
            }
        ],
        model = "gpt-3.5-turbo",
    )

    # Extract the output
    accessionNumberValue = accessionNumber.choices[0].message.content


    ## Continent
    # Set continent as either "Asia" or "Africa", depending on the project.
    # continentValue = "Asia"
    # continentValue = "Africa"


    ## Country
    country = client.chat.completions.create(
        messages = [
            {
                "role": "user",
                "content": f"Here is label text from an herbarium specimen: '{translated_text}'. In what country was this herbarium specimen collected? Return only the country and no extraneous text.",
            }
        ],
        model = "gpt-3.5-turbo",
    )

    # Extract the output
    countryValue = country.choices[0].message.content


    ## Locality Description
    localityDescription = client.chat.completions.create(
        messages = [
            {
                "role": "user",
                "content": f"Here is label text from an herbarium specimen: '{translated_text}'. What is the locality description for where this herbarium specimen was collected? Return only the locality description and no extraneous text.",
                "temperature": 0.2,
            }
        ],
        model = "gpt-3.5-turbo",
    )

    # Extract the output
    localityDescriptionValue = localityDescription.choices[0].message.content
    

    ## Elevation
    elevation = client.chat.completions.create(
        messages = [
            {
                "role": "user",
                "content": f"Here is label text from an herbarium specimen: '{translated_text}'. What elevation was this herbarium specimen collected at? Return only the elevation with units and no extraneous text.",
            }
        ],
        model = "gpt-3.5-turbo",
    )

    # Extract the output
    elevationValue = elevation.choices[0].message.content


    ## Collection Day
    collectionDay = client.chat.completions.create(
        messages = [
            {
                "role": "user",
                "content": f"Here is label text from an herbarium specimen: '{translated_text}'. What is the day of month that this herbarium specimen was collected on? Return only the day of month as a numeric digit, with no extraneous text.",
            }
        ],
        model = "gpt-3.5-turbo",
    )

    # Extract the output
    collectionDayValue = collectionDay.choices[0].message.content


    ## Collection Month
    collectionMonth = client.chat.completions.create(
        messages = [
            {
                "role": "user",
                "content": f"Here is label text from an herbarium specimen: '{translated_text}'. What is the month that this herbarium specimen was collected in? Return only the month as a numeric digit from 1 to 12, with no extraneous text.",
            }
        ],
        model = "gpt-3.5-turbo",
    )

    # Extract the output
    collectionMonthValue = collectionMonth.choices[0].message.content


    ## Collection Year
    collectionYear = client.chat.completions.create(
        messages = [
            {
                "role": "user",
                "content": f"Here is label text from an herbarium specimen: '{translated_text}'. What is the year that this herbarium specimen was collected in? Return only the year as a four digit number, with no extraneous text.",
            }
        ],
        model = "gpt-3.5-turbo",
    )

    # Extract the output
    collectionYearValue = collectionYear.choices[0].message.content


    ## Create a new row for the results data frame
    new_row = pd.DataFrame({'barcodeID': [barcode_id], 'AccessionNumber': [accessionNumberValue], 'Continent': [continentValue], 'Country': [countryValue], 
                            'LocalityDescription': [localityDescriptionValue], 'Elevation': [elevationValue], 'CollectionDay': [collectionDayValue], 
                            'CollectionMonth': [collectionMonthValue], 'CollectionYear': [collectionYearValue]})
    
    # Append the new row to the results data frame
    results = pd.concat([results, new_row], ignore_index=True)

    # Print the row that was processed
    print(f"{index}")

Print the results.

In [ ]:
results

# Step 3: Formatting for Tropicos

This section formats the data in the results data frame for upload to Tropicos.org, the Missouri Botancial Garden's database for storing specimen records. Accordingly, this section is tailored specifically for Tropicos upload, and would need to be modified for use with other databases.

Load the following libraries.

In [ ]:
import pandas as pd
from fuzzywuzzy import fuzz
from fuzzywuzzy import process
import re
import numpy as np
import datetime

Save results as GPTsheetsOutput.

In [ ]:
GPTsheetsOutput = results

Initialize an empty data frame that contains all the columns required for bulk specimen upload to Tropicos.

In [ ]:
specimenBulkData = pd.DataFrame(columns=["Authority", "AuthorityKey", "CollectorString", "SeniorCollectorPersonID", "CollectionNumber", "CollectionEventID", 
                                         "EventNumber", "DeterminationQualifier", "DeterminedBy", "DeterminedByPersonID", "DeterminationDay", "DeterminationMonth", 
                                         "DeterminationYear", "DeterminationInstitution", "LocationID", "MinimumLatitude", "MinimumLongitude", "MaximumLatitude", 
                                         "MaximumLongitude", "CoordinateMethod", "MinimumElevation", "MaximumElevation", "ElevationUnit", "ElevationMethod", 
                                         "UncertaintyMeters", "Sources", "CoordinateNote", "MinimumDay", "MinimumMonth", "MinimumYear", "MaximumDay", "MaximumMonth", 
                                         "MaximumYear", "LocalityNote", "DescriptionNote", "HabitatNote", "VegetationDescription", "GeneralNote", "Duplicates", 
                                         "IsCultivated", "FooterID", "Institutions", "DateLanguage", "OtherCollectorIDs", "GeographicKeywords", "GeolocationNote", 
                                         "ConditionEcologicalValues", "TownshipRangeNote", "Barcodes", "GeneralKeywords", "PreviousDeterminationIDs", 
                                         "ExsiccataePublicationID", "ExsiccataeNumber", "ScanningProjectCode"])

In Tropicos, locations are assigned unique identifiers. These identifiers must be included in specimen data that is bulk uploaded to Tropicos. In preparation for fuzzy matching of ChatGPT output to Tropicos identifers, load the Tropicos location list.

In [ ]:
LocationsDataDumpPath = "path to file with list of location identifiers"
LocationsDataDump = pd.read_csv(LocationsDataDumpPath, delimiter = '\t')

Define the following functions.

In [ ]:
# Define a 'find_matching_id' function for fuzzy matching
def find_matching_id(ocr_output, tropicos_data, id_column):
    # Find the string that best matches the OCR output
    match = process.extractOne(ocr_output, tropicos_data, scorer=fuzz.token_sort_ratio)
    
    # Extract the matched string and its index
    matched_string, score, index = match
    
    # Use the index to find the corresponding identifier
    matched_id = id_column.iloc[index]
    
    return matched_id

# Define a 'has_numbers' function that returns TRUE if a string contains any numbers and FALSE if it does not
def has_numbers(inputString):
    inputString = str(inputString)
    return any(char.isdigit() for char in inputString)

# Define a 'contains_only_digits' function that returns TRUE if a string contains only digits and FALSE if it contains any characters that are not digits
def contains_only_digits(input_string):
    for char in input_string:
        if not char.isdigit():
            return False
    return True

# Define an 'extract_elevation_info' function that extracts numeric elevation values and units, including their ranges
def extract_elevation_info(elevation_str):
    patternElevation = r'(\d+(\.\d+)?)-?(\d+(\.\d+)?)?\s*([mf]?)'
    matchElevation = re.search(patternElevation, elevation_str, re.I)
    
    if matchElevation:
        min_value_str, _, max_value_str, _, unit_str = matchElevation.groups()
        
        min_value = float(min_value_str) if min_value_str else None
        max_value = float(max_value_str) if max_value_str else None

        # Ensure the values are within the specified range
        if min_value is not None and (min_value < -2000 or min_value > 30000):
            min_value = None
        if max_value is not None and (max_value < -2000 or max_value > 30000):
            max_value = None

        # Check if unit is present
        unit = unit_str.lower() if unit_str else ''

        # Convert to integers only if they are valid numbers
        if min_value is not None:
            min_value = int(min_value)
        if max_value is not None:
            max_value = int(max_value)

        # Return the results with empty strings if values are None
        if min_value is not None and max_value is not None:
            return min_value, max_value, unit
        elif min_value is not None:
            return min_value, '', unit
        elif max_value is not None:
            return '', max_value, unit
    
    return '', '', ''

# Define a 'combine_location_fields' function that creates a string of continent and country from the location data dump
def combine_location_fields(row):
    fields = [row['ContinentName'], row['CountryName']]
    non_nan_fields = [str(field) for field in fields if not pd.isna(field)]
    return ' '.join(non_nan_fields)

Clean the location data in GPTsheetsOutput.

In [ ]:
# Reset the index
GPTsheetsOutput.reset_index(drop=True, inplace=True)

# Replace 'USA' with 'United States' in the 'Continent' column
GPTsheetsOutput['Country'] = GPTsheetsOutput['Country'].replace('USA', 'United States')

Perform general cleaning on the entire GPTsheetsOutput spreadsheet.

In [ ]:
# Define a 'clean_cell' function that applies cleaning rules to a single cell
def clean_cell(cell):
    if isinstance(cell, str):
        original_cell = cell 
        cell = cell.lower()
        if cell in ['#ref!', '#error!']:
            return ''
        if any(keyword in cell for keyword in [' no ', ' not ', 'no ', 'not ', 'unknown', 'sorry', 'apologies', 'apologize', 'unable']):
            return ''
        return original_cell 
    return cell

# Apply the 'clean_cell' function to each cell
GPTsheetsOutput = GPTsheetsOutput.map(clean_cell)

# Replace missing values with ''
GPTsheetsOutput = GPTsheetsOutput.fillna('')

Run a loop that uses data in GPTsheetsOutput to add a row per specimen to specimenBulkData.

In [ ]:
# Create an empty list to store the rows of data
rows_to_append = []

for currentRow in range(len(GPTsheetsOutput)):

    # Project code
    # Set the current project code as either "ASIA" or "AFRICA".
    # currentProjectCode = "ASIA"
    # currentProjectCode = "AFRICA"
    
    # Collector
    # Set collector as a generic collector for all specimens.
    currentCollectorID = '100152366'
    currentCollectorNameTropicos = 'AI_COLLECTOR, *'
    

    # Location fuzzy matching
    # Only set location to country. If country can't be found, set to continent.

    # Continent
    currentContinentNameOCR = str(GPTsheetsOutput.loc[currentRow, 'Continent'])

    if (currentContinentNameOCR == ''):
        currentContinentID = ''
        currentContinentNameTropicos = ''
        fuzzyScoreContinentName = 0
    else:
        # Use fuzzy matching to find the continent and its Tropicos ID
        currentContinentID = find_matching_id(currentContinentNameOCR, LocationsDataDump['ContinentName'], LocationsDataDump['ContinentID'])
    
        # Use the currentContinentID to find the current continent name
        # Check if there are any matching rows
        matching_rows_continent = LocationsDataDump[LocationsDataDump['ContinentID'] == currentContinentID]

        if not matching_rows_continent.empty:
            currentContinentNameTropicos = matching_rows_continent['ContinentName'].values[0]
            # Obtain 'fuzzy matching score' for continent (100 is perfect match, 0 is no match)
            fuzzyScoreContinentName = fuzz.token_set_ratio(currentContinentNameOCR, currentContinentNameTropicos)
        else:
            # Handle the case where there are no matching rows
            currentContinentNameTropicos = ''
            fuzzyScoreContinentName = 0
        
    
    # Country
    currentCountryNameOCR = str(GPTsheetsOutput.loc[currentRow, 'Country'])

    if (currentCountryNameOCR == ''):
        currentCountryID = ''
        currentCountryNameTropicos = ''
        fuzzyScoreCountryName = 0
    else:
        # Use fuzzy matching to find the country and its Tropicos ID
        currentCountryID = find_matching_id(currentCountryNameOCR, LocationsDataDump['CountryName'], LocationsDataDump['CountryID'])
    
        # Use the currentCountryID to find the current country name
        # Check if there are any matching rows
        matching_rows_country = LocationsDataDump[LocationsDataDump['CountryID'] == currentCountryID]

        if not matching_rows_country.empty:
            currentCountryNameTropicos = matching_rows_country['CountryName'].values[0]
            # Obtain 'fuzzy matching score' for country (100 is perfect match, 0 is no match)
            fuzzyScoreCountryName = fuzz.token_set_ratio(currentCountryNameOCR, currentCountryNameTropicos)
        else:
            # Handle the case where there are no matching rows
            currentCountryNameTropicos = ''
            fuzzyScoreCountryName = 0


    # Use the currentContinentID and currentCountryID to find the location ID.              
    filter_conditions = (
        ((LocationsDataDump['ContinentID'] == currentContinentID) & (fuzzyScoreContinentName > 75)) &
        ((LocationsDataDump['CountryID'] == currentCountryID) & (fuzzyScoreCountryName > 75))
    )   
    if filter_conditions.any():
        currentLocationID = LocationsDataDump.loc[filter_conditions, 'LocationID'].values[0]
    else:
        # If no result is found, try continent
        filter_conditions = (
            ((LocationsDataDump['ContinentID'] == currentContinentID) & (fuzzyScoreContinentName > 75))
        )  
        if filter_conditions.any():
            currentLocationID = LocationsDataDump.loc[filter_conditions, 'LocationID'].values[0]
        else:
            # If no result is found, return an emtpy string
            currentLocationID = ""
            

    # Accession number
    # If the accession number is 6 or more digits in length, set accession number as the verbatim accession number,
    # if not, set accession number as blank.
    if (len(GPTsheetsOutput.loc[currentRow, 'AccessionNumber']) >= 6):
        currentAccessionNumber = GPTsheetsOutput.loc[currentRow, 'AccessionNumber']
    else:
        currentAccessionNumber = ""

    # If the accession number contains any characters aside from numeric digits, set it to an empty string
    if (contains_only_digits(currentAccessionNumber) == False):
        currentAccessionNumber = ''


    # Collection number
    # Set the collection number for every specimen to 's.n.'. This will be updated during manual editing.
    currentCollectionNumber = 's.n.'
    

    # For the barcode field in Tropicos, create a list that contains the following, each separated by a comma and surrounded by parentheses:
    # institution, barcode, accession number, barcode object (e.g., (MO,MO-1859342,3794786,sheet)
    currentBarcodeID = GPTsheetsOutput.loc[currentRow, 'barcodeID']
    currentBarcodeList = f"(MO,{currentBarcodeID},{currentAccessionNumber},sheet)"
    

    # Create objects for the day, month, and year of collection.
    # Return no value for invalid dates and remove non-numeric text

    # Collection year:
    # If CollectionYear is 4 digits, return this as currentCollectionYear
    if (len(str(GPTsheetsOutput.loc[currentRow, 'CollectionYear'])) == 4):
        currentCollectionYear = GPTsheetsOutput.loc[currentRow, 'CollectionYear']
    elif (has_numbers(GPTsheetsOutput.loc[currentRow, 'CollectionYear']) == True):
        pattern = r'is (\d+)' 
        match = re.search(pattern, str(GPTsheetsOutput.loc[currentRow, 'CollectionYear']))
        if match:
            currentCollectionYear = match.group(1)
        else:
            currentCollectionYear = ""
    else:
        currentCollectionYear = "" 

    # If currentCollectionYear is not an integer (i.e., contains 4 characters, but not all are digits) set currentCollectionYear as an empty string
    if currentCollectionYear.isdigit():
        currentCollectionYear = int(currentCollectionYear)
    else:
        currentCollectionYear = ''
    
    # If currentCollectionYear is 0, make currentCollectionYear have no value
    if currentCollectionYear != '': 
        if currentCollectionYear == 0:
            currentCollectionYear = ""
    else:
        currentCollectionYear = ""

    # Check if currentCollectionYear contains non-numeric characters, and if so, set it to an empty string
    if (currentCollectionYear == ''):
        currentCollectionYear = ''
    elif not re.match(r'^\d+$', str(currentCollectionYear)):
        currentCollectionYear = ''
    else:
        currentCollectionYear = currentCollectionYear
    
    # If currentCollectionYear is not within 1850-2030, set currentCollectionYear to empty string
    if (currentCollectionYear == ''):
        currentCollectionYear = ''
    elif ((1850 <= int(currentCollectionYear) <= 2030) == False):
        currentCollectionYear = ''
    else:
        currentCollectionYear = currentCollectionYear
    
    
    # Collection month:
    # If CollectionMonth is less than 3 digits, return this as currentCollectionMonth
    if (len(str(GPTsheetsOutput.loc[currentRow, 'CollectionMonth'])) < 3): 
        currentCollectionMonth = GPTsheetsOutput.loc[currentRow, 'CollectionMonth']
    elif (has_numbers(GPTsheetsOutput.loc[currentRow, 'CollectionMonth']) == True):
        pattern = r'is (\d+)'
        match = re.search(pattern, str(GPTsheetsOutput.loc[currentRow, 'CollectionMonth']))
        if match:
            currentCollectionMonth = match.group(1)
            if currentCollectionMonth.isdigit():
                currentCollectionMonth = int(currentCollectionMonth)
            else:
                currentCollectionMonth = "" 
        else:
            currentCollectionMonth = ""  
    else:
        currentCollectionMonth = ""

    if currentCollectionMonth != '':
        if isinstance(currentCollectionMonth, str) and currentCollectionMonth.isdigit():
            currentCollectionMonth = int(currentCollectionMonth)
        else:
            currentCollectionMonth = "" 
        if currentCollectionMonth == 0:
            currentCollectionMonth = ""
    else:
        currentCollectionMonth = ""

    # Check if currentCollectionMonth contains non-numeric characters, and if so, set it to an empty string
    if (currentCollectionMonth == ''):
        currentCollectionMonth = ''
    elif not re.match(r'^\d+$', str(currentCollectionMonth)):
        currentCollectionMonth = ''
    else:
        currentCollectionMonth = currentCollectionMonth
    
    # If currentCollectionMonth is not within 1-12, set currentCollectionMonth to empty string
    if (currentCollectionMonth == ''):
        currentCollectionMonth = ''
    elif ((1 <= int(currentCollectionMonth) <= 12) == False):
        currentCollectionMonth = ''
    else:
        currentCollectionMonth = currentCollectionMonth
        
    
    # Collection day:
    # If there is no currentCollectionMonth, return no value for currentCollectionDay
    if (has_numbers(currentCollectionMonth) == False): 
        currentCollectionDay = ""
    # If CollectionDay is less than 3 digits, return this as currentCollectionDay
    elif (len(str(GPTsheetsOutput.loc[currentRow, 'CollectionDay'])) < 3):
        currentCollectionDay = GPTsheetsOutput.loc[currentRow, 'CollectionDay']
    elif (has_numbers(GPTsheetsOutput.loc[currentRow, 'CollectionDay']) == True):
        pattern = r'is (\d+)' 
        match = re.search(pattern, str(GPTsheetsOutput.loc[currentRow, 'CollectionDay']))
        if match:
            currentCollectionDay = match.group(1)
            if currentCollectionDay.isdigit():
                currentCollectionDay = int(currentCollectionDay)
            else:
                currentCollectionDay = ""  
        else:
            currentCollectionDay = ""  
    else:
        currentCollectionDay = ""

    # Check if currentCollectionDay contains non-numeric characters, and if so, set it to an empty string
    if (currentCollectionDay == ''):
        currentCollectionDay = ''
    elif not re.match(r'^\d+$', str(currentCollectionDay)):
        currentCollectionDay = ''
    else:
        currentCollectionDay = currentCollectionDay

    # If currentCollectionDay is not within 1-31, set currentCollectionDay to empty string
    if (currentCollectionDay == ''):
        currentCollectionDay = ''
    elif ((1 <= int(currentCollectionDay) <= 31) == False):
        currentCollectionDay = ''
    else:
        currentCollectionDay = currentCollectionDay


    # Create objects for elevation and the unit of elevation
    currentElevationInfo = extract_elevation_info(GPTsheetsOutput.loc[currentRow, 'Elevation'])
    currentMinimumElevation = currentElevationInfo[0]
    currentMaximumElevation = currentElevationInfo[1]
    currentElevationUnit = currentElevationInfo[2]

    # If currentMinimumElevation is not within -2000-6000, set currentMinimumElevation to empty string
    if (currentMinimumElevation == ''):
        currentMinimumElevation = ''
    elif ((currentElevationUnit == 'm') and (-2000 <= int(currentMinimumElevation) <= 6000) == False):
        currentMinimumElevation = ''
    else:
        currentMinimumElevation = currentMinimumElevation

    # If currentMaximumElevation is not within -2000-6000, set currentMaximumElevation to empty string
    if (currentMaximumElevation == ''):
        currentMaximumElevation = ''
    elif ((currentElevationUnit == 'm') and (-2000 <= int(currentMaximumElevation) <= 6000) == False):
        currentMaximumElevation = ''
    else:
        currentMaximumElevation = currentMaximumElevation

    
    # Create object for LocalityNote
    # Use the verbatim text from the GPT-Sheets output for these fields, unless an error occurred or no note is given.
    currentLocalityNote = GPTsheetsOutput.loc[currentRow, 'LocalityDescription']

    # Set the currentAuthority to 'missouri botanical garden'
    currentAuthority = 'missouri botanical garden'

    # Set currentAuthorityKey to 'OCR' followed by a unique number, all followed by the project code and today's date
    today_date = datetime.datetime.now().strftime("%Y%m%d")
    currentAuthorityKey = 'OCR' + str(currentRow) + "_" + currentProjectCode + "_" + today_date + "a"

    
    # Create a new row to add to specimenBulkData
    newRow = {
        "Authority": currentAuthority, 
        "AuthorityKey": currentAuthorityKey, 
        "CollectorString": currentCollectorNameTropicos, 
        "SeniorCollectorPersonID": currentCollectorID, 
        "CollectionNumber": currentCollectionNumber, 
        "CollectionEventID": "",                        
        "EventNumber": "", 
        "DeterminationQualifier": "", 
        "DeterminedBy": "", 
        "DeterminedByPersonID": "",
        "DeterminationDay": "", 
        "DeterminationMonth": "", 
        "DeterminationYear": "", 
        "DeterminationInstitution": "", 
        "LocationID": currentLocationID, 
        "MinimumLatitude": "", 
        "MinimumLongitude": "", 
        "MaximumLatitude": "", 
        "MaximumLongitude": "", 
        "CoordinateMethod": "", 
        "MinimumElevation": currentMinimumElevation, 
        "MaximumElevation": currentMaximumElevation, 
        "ElevationUnit": currentElevationUnit, 
        "ElevationMethod": "", 
        "UncertaintyMeters": "", 
        "Sources": "", 
        "CoordinateNote": "", 
        "MinimumDay": currentCollectionDay, 
        "MinimumMonth": currentCollectionMonth, 
        "MinimumYear": currentCollectionYear, 
        "MaximumDay": "", 
        "MaximumMonth": "", 
        "MaximumYear": "", 
        "LocalityNote": currentLocalityNote, 
        "DescriptionNote": "", 
        "HabitatNote": "", 
        "VegetationDescription": "", 
        "GeneralNote": "Tropicos transcription obtained using Google Cloud OCR and GPT 3.5 Turbo, followed by human editing.", 
        "Duplicates": "", 
        "IsCultivated": "", 
        "FooterID": "", 
        "Institutions": "MO", 
        "DateLanguage": "", 
        "OtherCollectorIDs": "",
        "GeographicKeywords": "", 
        "GeolocationNote": "", 
        "ConditionEcologicalValues": "", 
        "TownshipRangeNote": "", 
        "Barcodes": currentBarcodeList, 
        "GeneralKeywords": "", 
        "PreviousDeterminationIDs": "", 
        "ExsiccataePublicationID": "", 
        "ExsiccataeNumber": "",
        "ScanningProjectCode": currentProjectCode,
    }

    # Append the `newRow`
    rows_to_append.append(newRow)

    # Print currentRow
    print({currentRow})

Combine data frames outside the loop

In [ ]:
# Convert each dictionary to a data frame
dataframes_to_concat = [pd.DataFrame([newRow]) for newRow in rows_to_append]

# Concatenate all the data frames outside the loop
specimenBulkData = pd.concat([specimenBulkData] + dataframes_to_concat, ignore_index=True)

Replace 'NaN' with '' in specimenBulkData

In [ ]:
specimenBulkData = specimenBulkData.fillna('')

Look at specimenBulkData

In [ ]:
pd.set_option('display.max_columns', None)

specimenBulkData

Remove text from cells with non-English characters and replace line breaks with spaces.

In [ ]:
def clean_cell(cell):
    if isinstance(cell, str):
        # Replace line breaks with spaces
        cell = cell.replace('\n', ' ')
        # Replace non-English characters with an empty string
        if re.search(r'[^\x00-\x7F]', cell):
            return ''
    return cell

specimenBulkData = specimenBulkData.applymap(clean_cell)

Write specimenBulkData to a CSV for bulk upload to Tropicos.

In [ ]:
specimenBulkData.to_csv(f'{output_folder}/{csv_file_name}', index=False)

Write specimenBulkData to a tab delimited text file for bulk upload to Tropicos.

In [ ]:
specimenBulkData.to_csv(f'{output_folder}/{txt_file_name}', sep='\t', index=False)